In [ ]:
import os
import json
from datetime import datetime, timezone, timedelta
import time
import uuid
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import WMLCredentialsCP4D
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import wos_sdk_utils as wos_util
import wml_sdk_utils as wml_util

# OpenScale Configuration

In [ ]:
# SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
# SUBSCRIPTION_NAME = "DeepLIIF Monitor wendy 2"

# MODEL_ASSET_ID = "3956ba10-52f9-40f8-8769-693a4a3ddbc5"
# WML_SPACE_ID = '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7' 

# WOS_GUID = '00000000-0000-0000-0000-000000000000'

In [ ]:
SERVICE_PROVIDER_NAME = os.environ['SERVICE_PROVIDER_NAME']
SUBSCRIPTION_NAME = os.environ['SUBSCRIPTION_NAME']

MODEL_ASSET_ID = os.environ['MODEL_ASSET_ID']
WML_SPACE_ID = os.environ['WML_SPACE_ID']

WOS_GUID = os.environ['WOS_GUID']

In [ ]:
wos_client = wos_util.get_client()
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)

In [ ]:
metadata_deployment = wml_util.metadata_yml_load(wml_client,'deployment')[MODEL_ASSET_ID]
metadata_monitor = wml_util.metadata_yml_load(wml_client,'monitor')

## 1. Subscribe to Deployment
Here we create a dummy subscription because there is no need for OpenScale to communicate with the deployment's scoring endpoint.

In [ ]:
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest

In [ ]:
existing_providers_dict = wos_client.service_providers.list().get_result().to_dict()['service_providers']
existing_providers = [sp['entity']['name'] for sp in existing_providers_dict]

SERVICE_PROVIDER_ID = next((sp['metadata']['id'] for sp in existing_providers_dict \
                            if sp['entity']['name'] == SERVICE_PROVIDER_NAME))
print(f"Service provider ID: {SERVICE_PROVIDER_ID}")

In [ ]:
wos_client.service_providers.show()

In [ ]:
existing_subscriptions_dict = wos_client.subscriptions.list().get_result().to_dict()['subscriptions']
existing_subscriptions = [sp['entity']['asset']['name'] for sp in existing_subscriptions_dict]

if not SUBSCRIPTION_NAME in existing_subscriptions:
    ASSET_ID = str(uuid.uuid4())
    ASSET_NAME = SUBSCRIPTION_NAME
    url = ''

    ASSET_DEPLOYMENT_ID = str(uuid.uuid4())
    ASSET_DEPLOYMENT_NAME = SUBSCRIPTION_NAME
    
    subscription_details = wos_client.subscriptions.add(
        data_mart_id=WOS_GUID,
        service_provider_id=SERVICE_PROVIDER_ID,
        asset=Asset(
            asset_id=ASSET_ID,
            name=ASSET_NAME,
            url=url,
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.MULTICLASS_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=ASSET_DEPLOYMENT_ID,
            name=ASSET_DEPLOYMENT_NAME,
            deployment_type= DeploymentTypes.ONLINE
        ),
        asset_properties=AssetPropertiesRequest(
            probability_fields=['probability']
            )
    ).result
    
    SUBSCRIPTION_ID = subscription_details.metadata.id
    print("Subscription ID: {}".format(SUBSCRIPTION_ID))
else:
    SUBSCRIPTION_ID = next((sp['metadata']['id'] for sp in existing_subscriptions_dict \
                            if sp['entity']['asset']['name'] == SUBSCRIPTION_NAME))
    print("Subscription ID: {}".format(SUBSCRIPTION_ID))

In [ ]:
wos_client.subscriptions.show()

## 2. Update Deployment Metadata File

In [ ]:
metadata_update = {MODEL_ASSET_ID:{'openscale_subscription_id':SUBSCRIPTION_ID}}

In [ ]:
wml_util.metadata_yml_update(metadata_update,wml_client)

In [ ]:
metadata_deployment = wml_util.metadata_yml_load(wml_client)[MODEL_ASSET_ID]
metadata_deployment

## 3. Configure Custom Metric Monitors

### Check existence of custom monitor instance

In [ ]:
metadata_deployment['openscale_custom_metric_provider'].keys()

In [ ]:
for monitor_id in metadata_deployment['openscale_custom_metric_provider'].keys():
    print('*'*20,'checking',monitor_id,'*'*20)
    print(wos_util.get_monitor_instance(monitor_id,SUBSCRIPTION_ID,wos_client))

In [ ]:
# Openscale MRM service invokes custom metrics deployment url during runtime and wait for the default time of 60 second's to 
# to check the run status ie finished/Failed and fetch the latest measurement. Increase the wait time, if the runtime deployment 
# takes more than 60 seconds to compute and publish the custom metrics 

#Update the wait time here.
custom_metrics_wait_time = 360 #time in seconds <update the time here>

### Create one custom monitor instance per monitor

In [ ]:
for monitor_id in metadata_deployment['openscale_custom_metric_provider'].keys():
    monitor_instance_details = wos_util.monitor_instance_create(monitor_id,
                                                                metadata_deployment,
                                                                metadata_monitor,
                                                                custom_metrics_wait_time,
                                                                wos_client)
    print(monitor_instance_details)

### Manual Evaluation

In [ ]:
import uuid
from pprint import pprint

subscription_id = SUBSCRIPTION_ID
for monitor_id in metadata_deployment['openscale_custom_metric_provider'].keys():
    print('*'*30,monitor_id,'*'*30)
    parameters = {
        "custom_metrics_provider_id": metadata_monitor[monitor_id]['integrated_system_id'],
        "custom_metrics_wait_time":   custom_metrics_wait_time,
        "run_details": {
        "run_id": str(uuid.uuid4()),
        "run_status": "Running"
        }
    }

    payload= {
        "data_mart_id" : WOS_GUID,
        "subscription_id" : subscription_id,
        "custom_monitor_id" : monitor_id,
        "custom_monitor_instance_id" : wos_util.get_monitor_instance(monitor_id,subscription_id,wos_client)['metadata']['id'],
        "custom_monitor_instance_params": parameters

    }

    input_data= { "input_data": [ { "values": payload } ]
                }

#     print(input_data)

    job_details = wml_client.deployments.score(metadata_monitor[monitor_id]['wml_deployment_id'], input_data)
    pprint(job_details)